## CIIC5018 / ICOM5018
## Network Security and Cryptography
## Project 7: RSA Implementation
### Francis Jose Patron Fidalgo (802180833)
### sec: 060
### 11/08/2022

# 1) modular exponentiation function

In [301]:
def mod_exp_r2l(base, exponent, modulus):
    if modulus == 1:
        return 0
    result = 1
    base %= modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent >>= 1
        base = (base * base) % modulus
    return result

# 2) Miller-Rabin algorithm

In [302]:
from random import randrange, getrandbits

INCONCLUSIVE = 'inconclusive'
COMPOSITE = 'composite'

def miller_rabin(n):
    if n == 2:
        return INCONCLUSIVE
    if n % 2 == 0:
        return COMPOSITE
    q = n - 1
    k = 0
    while q % 2 == 0:
        q = q // 2
        k += 1
    a = randrange(2, n - 1)
    tmp = pow(a, q, n) # a^q mod n
    if tmp == 1 or tmp == n - 1:
        return INCONCLUSIVE
    for _ in range(k - 1):
        tmp = pow(tmp, 2, n)
        if tmp == n - 1:
            return INCONCLUSIVE
    return COMPOSITE

In [303]:
def is_prime(n):
    for _ in range(20):
        if miller_rabin(n) is COMPOSITE:
            return False
    return True

In [304]:
def gen_prime(n):
    n += 1
    while not is_prime(n):
        n+=1
    return n

# 3) RSA

In [305]:
def euclid(a, b):
    if (b == 0):
        return a
    else:
        return euclid(b, a % b)

def ext_euclid(a, b):
    # Base Case
    if a == 0 :
        return b,0,1
             
    gcd,x1,y1 = ext_euclid(b%a, a)
     
    # Update x and y using results of recursive
    # call
    x = y1 - (b//a) * x1
    y = x1
     
    return gcd,x,y

In [306]:
def gen_keys():
    # generate p and q 8-byte prime numbers
    p = gen_prime(getrandbits(64))
    q = gen_prime(getrandbits(64))

    # calculate n
    n = p * q

    # calculate phi(n) 
    pn = (p-1) * (q - 1)

    # select e such that its relaively prime to pn
    e = randrange(1, pn)
    while euclid(e, pn) != 1:
        e = randrange(1, pn)

    # determine d
    d, x, y = ext_euclid(e, pn)
    d = x % pn

    return {
        'public': (n, e),
        'private': (n, d)
    }

In [307]:
def encrypt(pub_key, m):
    return mod_exp_r2l(m, pub_key[1], pub_key[0])

In [308]:
def decrypt(priv_key, c):
    return mod_exp_r2l(c, priv_key[1], priv_key[0])

# Tests

### 1) Miller-Rabin algorithm

In [ ]:
def test_miller_rabin():